# 🎨 Trellis NIM - Simple Deployment & Inference

A streamlined notebook for deploying NVIDIA Trellis NIM and generating 3D models from text prompts.

## Steps:
1. **GPU Check** - Verify L40s GPU availability
2. **Deploy NIM** - Run the Trellis NIM container
3. **Generate 3D Models** - Simple prompt-based generation

---


## Step 1: Check GPU Availability

Verify that we have access to an L40s GPU:


In [ ]:
!nvidia-smi


## Step 2: Deploy Trellis NIM Container

**⚠️ IMPORTANT**: Replace `<PASTE_API_KEY_HERE>` with your actual NGC API key from https://ngc.nvidia.com/setup/api-key


In [ ]:
# Set your NGC API key here
import os
os.environ['NGC_API_KEY'] = '<PASTE_API_KEY_HERE>'  # Replace with your actual API key

# Verify the API key is set
if os.environ['NGC_API_KEY'] == '<PASTE_API_KEY_HERE>':
    print('❌ Please replace <PASTE_API_KEY_HERE> with your actual NGC API key!')
else:
    print('✅ NGC API key is set')


In [ ]:
# Docker login to NGC registry
!echo "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin


In [ ]:
# Create cache directory
!mkdir -p ~/.cache/nim
!chmod 777 ~/.cache/nim
!echo "Cache directory created at: ~/.cache/nim"


In [ ]:
# Start Trellis NIM container (this will run in the background)
# Note: This may take several minutes to download and start the first time

!docker run -d --name=nim-server \
  --runtime=nvidia --gpus='"device=0"' \
  -e NGC_API_KEY=$NGC_API_KEY \
  -p 8000:8000 \
  -v ~/.cache/nim:/opt/nim/.cache/ \
  nvcr.io/nim/microsoft/trellis:latest

print('🚀 Trellis NIM container started!')
print('⏳ Please wait 2-5 minutes for the container to fully initialize...')


In [ ]:
# Check container status
!docker ps | grep nim-server
!echo "\n📊 Container logs (last 10 lines):"
!docker logs nim-server --tail 10


## Step 3: Generate 3D Models

Now you can generate 3D models by simply changing the prompt below and running the cell:


In [ ]:
#CHANGE THIS PROMPT TO GENERATE DIFFERENT 3D MODELS. BRANDS OR SPECIFIC PRODUCTS WILL FAIL THE GENERATION. 
PROMPT = "A simple coffee shop interior"

#Change output filename
OUTPUT_FILE = "result.glb"

# Optional: Change the seed for different variations (0 = random)
SEED = 0

# First, let's test if the NIM container is responding
print("🔍 Testing NIM container connectivity...")
health_check = !curl -s http://localhost:8000/health || curl -s http://localhost:8000/ || echo "CONNECTION_FAILED"

if "CONNECTION_FAILED" in ' '.join(health_check):
    print("❌ Cannot connect to NIM container at http://localhost:8000")
    print("💡 Make sure the container is running with: docker ps | grep nim-server")
    print("💡 Check container logs with: docker logs nim-server")
else:
    print("✅ NIM container is responding")

print("\n📡 Sending generation request to Trellis NIM...")

# Use the exact curl command format from your original request
import subprocess
import json
import base64

# Create the JSON payload
json_payload = json.dumps({
    "prompt": PROMPT,
    "seed": SEED
})

# Execute curl command exactly as specified
curl_cmd = [
    'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
    '-H', 'Accept: application/json',
    '-H', 'Content-Type: application/json',
    '-d', json_payload,
    '--silent'
]

print(f"🎯 Generating: '{PROMPT}'")
print("⏳ This may take 1-3 minutes...")

try:
    # Run the curl command
    result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode != 0:
        print(f"❌ Curl command failed with return code: {result.returncode}")
        print(f"Error: {result.stderr}")
        print(f"Command: {' '.join(curl_cmd)}")
    else:
        response_text = result.stdout.strip()
        print(f"📨 Raw response length: {len(response_text)} characters")
        print(f"📨 Response preview: {response_text[:300]}...")
        
        if not response_text:
            print("❌ Empty response from server")
        elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
            print("❌ Received HTML response instead of JSON - check if container is fully started")
        else:
            try:
                # Parse JSON response
                response_data = json.loads(response_text)
                print("✅ Valid JSON response received")
                
                # Check for artifacts in response
                if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                    artifact = response_data['artifacts'][0]
                    if 'base64' in artifact:
                        base64_data = artifact['base64']
                        
                        # Decode and save the GLB file
                        glb_data = base64.b64decode(base64_data)
                        
                        with open(OUTPUT_FILE, 'wb') as f:
                            f.write(glb_data)
                        
                        print(f"✅ SUCCESS! 3D model saved as: {OUTPUT_FILE}")
                        print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                        print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                    else:
                        print("❌ No base64 data in artifact")
                        print(f"Artifact keys: {list(artifact.keys())}")
                else:
                    print("❌ No artifacts found in response")
                    print(f"Response keys: {list(response_data.keys())}")
                    if 'error' in response_data:
                        print(f"Error from server: {response_data['error']}")
                        
            except json.JSONDecodeError as e:
                print(f"❌ Failed to parse JSON response: {e}")
                print(f"Raw response: {response_text}")

except subprocess.TimeoutExpired:
    print("❌ Request timed out (>5 minutes)")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


## 🎉 That's it!

### To generate more 3D models:
1. Go back to the cell above with `PROMPT = "..."` and `OUTPUT_FILE = "... .glb"`
2. Change the prompt to whatever you want
3. Run both cells again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`

### Useful commands:


In [ ]:
# Check if NIM container is running
!docker ps | grep nim-server


## Step 4: Launch Frontend UI

Now let's set up and launch the integrated frontend UI that will provide a web interface for interacting with your Trellis NIM service.


In [ ]:
# Clone the frontend UI repository
import os
import subprocess

# Check if the repository already exists
if os.path.exists('prompt-to-trellis'):
    print('📁 Frontend repository already exists')
else:
    print('📥 Cloning frontend repository...')
    result = subprocess.run(['git', 'clone', 'https://github.com/Soragi/prompt-to-trellis.git'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print('✅ Frontend repository cloned successfully')
    else:
        print(f'❌ Failed to clone repository: {result.stderr}')

# List the contents to see what we're working with
!ls -la prompt-to-trellis/


In [ ]:
# Examine the frontend structure and setup
import json
import shutil

print('🔍 Examining frontend structure...')
!find prompt-to-trellis -type f -name "*.html" -o -name "*.js" -o -name "*.css" -o -name "*.json" | head -10

# Check if it's a Node.js/npm project
if os.path.exists('prompt-to-trellis/package.json'):
    print('📦 Found package.json - This is a Node.js project')
    with open('prompt-to-trellis/package.json', 'r') as f:
        package_data = json.load(f)
    print(f'Project: {package_data.get("name", "Unknown")}')
    print(f'Description: {package_data.get("description", "No description")}')
    
    # Install dependencies if needed
    print('📥 Installing frontend dependencies...')
    !cd prompt-to-trellis && npm install
else:
    print('📄 Appears to be a static HTML/JS project')

# Check for main files
main_files = ['index.html', 'main.js', 'app.js', 'script.js']
found_files = []
for file in main_files:
    if os.path.exists(f'prompt-to-trellis/{file}'):
        found_files.append(file)

print(f'📋 Main files found: {found_files}')


In [ ]:
# Create API bridge configuration for the frontend
import json

# Create a configuration file that the frontend can use to connect to NIM
config = {
    "nim_endpoint": "http://localhost:8000",
    "api_base": "http://localhost:8000/v1",
    "infer_endpoint": "http://localhost:8000/v1/infer",
    "health_endpoint": "http://localhost:8000/health"
}

# Write config to the frontend directory
with open('prompt-to-trellis/nim-config.json', 'w') as f:
    json.dump(config, f, indent=2)

print('⚙️ Created NIM service configuration for frontend')

# Create a simple JavaScript module for NIM integration
nim_integration_js = '''
// NIM Service Integration
class NIMService {
    constructor() {
        this.config = null;
        this.loadConfig();
    }
    
    async loadConfig() {
        try {
            const response = await fetch('./nim-config.json');
            this.config = await response.json();
            console.log('NIM config loaded:', this.config);
        } catch (error) {
            console.error('Failed to load NIM config:', error);
            // Fallback config
            this.config = {
                nim_endpoint: "http://localhost:8000",
                infer_endpoint: "http://localhost:8000/v1/infer"
            };
        }
    }
    
    async generateModel(prompt, seed = 0) {
        if (!this.config) {
            await this.loadConfig();
        }
        
        const payload = {
            prompt: prompt,
            seed: seed
        };
        
        try {
            console.log('Sending request to NIM:', payload);
            const response = await fetch(this.config.infer_endpoint, {
                method: 'POST',
                headers: {
                    'Accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(payload)
            });
            
            if (!response.ok) {
                throw new Error(`HTTP error! status: ${response.status}`);
            }
            
            const data = await response.json();
            console.log('NIM response:', data);
            
            if (data.artifacts && data.artifacts.length > 0) {
                const artifact = data.artifacts[0];
                if (artifact.base64) {
                    return this.base64ToBlob(artifact.base64);
                }
            }
            
            throw new Error('No valid artifact in response');
            
        } catch (error) {
            console.error('NIM generation error:', error);
            throw error;
        }
    }
    
    base64ToBlob(base64Data) {
        const byteCharacters = atob(base64Data);
        const byteNumbers = new Array(byteCharacters.length);
        for (let i = 0; i < byteCharacters.length; i++) {
            byteNumbers[i] = byteCharacters.charCodeAt(i);
        }
        const byteArray = new Uint8Array(byteNumbers);
        return new Blob([byteArray], { type: 'model/gltf-binary' });
    }
    
    async checkHealth() {
        if (!this.config) {
            await this.loadConfig();
        }
        
        try {
            const response = await fetch(this.config.health_endpoint || this.config.nim_endpoint + '/health');
            return response.ok;
        } catch (error) {
            return false;
        }
    }
}

// Make NIMService available globally
window.NIMService = NIMService;
'''

# Write the NIM integration JavaScript
with open('prompt-to-trellis/nim-service.js', 'w') as f:
    f.write(nim_integration_js)

print('✅ Created NIM service JavaScript integration module')
print('📂 Files created:')
print('   - nim-config.json (configuration)')
print('   - nim-service.js (JavaScript integration)')

# Show the user what to do next
print('\\n🔧 To integrate with your frontend:')
print('1. Include nim-service.js in your HTML: <script src="nim-service.js"></script>')
print('2. Create a NIMService instance: const nim = new NIMService();')
print('3. Use nim.generateModel(prompt) to generate 3D models')
print('4. The returned blob can be used as a URL for 3D viewers')


In [ ]:
# Launch the frontend UI
import threading
import time
import webbrowser
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver
import os

def start_frontend_server(port=3000):
    """Start the frontend server in a separate thread"""
    class CORSHTTPRequestHandler(SimpleHTTPRequestHandler):
        def end_headers(self):
            self.send_header('Access-Control-Allow-Origin', '*')
            self.send_header('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
            self.send_header('Access-Control-Allow-Headers', 'Content-Type')
            super().end_headers()
        
        def do_OPTIONS(self):
            self.send_response(200)
            self.end_headers()
    
    os.chdir('prompt-to-trellis')
    
    try:
        with socketserver.TCPServer(("", port), CORSHTTPRequestHandler) as httpd:
            print(f"🌐 Frontend server started on http://localhost:{port}")
            httpd.serve_forever()
    except Exception as e:
        print(f"❌ Failed to start server on port {port}: {e}")

# Start the server in background
def launch_frontend():
    frontend_port = 3000
    
    # Check if the frontend files exist
    if not os.path.exists('prompt-to-trellis'):
        print("❌ Frontend directory not found. Please run the clone cell first.")
        return
    
    print("🚀 Starting frontend server...")
    
    # Start server in a separate thread
    server_thread = threading.Thread(target=start_frontend_server, args=(frontend_port,))
    server_thread.daemon = True
    server_thread.start()
    
    # Give the server a moment to start
    time.sleep(2)
    
    print(f"✅ Frontend UI is now available at: http://localhost:{frontend_port}")
    print("🎯 The UI is connected to your NIM service running on port 8000")
    print("💡 Use the web interface to generate 3D models with prompts")
    
    # Try to open the browser automatically
    try:
        webbrowser.open(f'http://localhost:{frontend_port}')
        print("🌐 Opening browser automatically...")
    except:
        print("🌐 Please manually open http://localhost:3000 in your browser")
    
    return f"http://localhost:{frontend_port}"

# Launch the frontend
frontend_url = launch_frontend()

# Display instructions
print("\\n" + "="*60)
print("🎉 FRONTEND UI LAUNCHED!")
print("="*60)
print(f"🔗 URL: {frontend_url}")
print("🔧 NIM Service: http://localhost:8000")
print("\\n📝 How to use:")
print("1. Open the URL above in your browser")
print("2. Enter a prompt in the text field")
print("3. Click generate to create a 3D model")
print("4. The .glb model will appear in the viewer on the right")
print("\\n💡 Example prompts:")
print("   • 'A modern chair'")
print("   • 'A wooden table'") 
print("   • 'A cozy bedroom'")
print("   • 'A futuristic car'")


In [ ]:
# Alternative: Launch with Node.js/npm if available
import subprocess

def launch_with_npm():
    """Launch frontend using npm if package.json exists"""
    if os.path.exists('prompt-to-trellis/package.json'):
        print("📦 Detected npm project, attempting to launch with npm...")
        try:
            os.chdir('prompt-to-trellis')
            
            # Try common npm start commands
            for cmd in ['npm start', 'npm run dev', 'npm run serve']:
                try:
                    print(f"🚀 Trying: {cmd}")
                    process = subprocess.Popen(cmd.split(), 
                                             stdout=subprocess.PIPE, 
                                             stderr=subprocess.PIPE)
                    time.sleep(3)  # Give it time to start
                    
                    if process.poll() is None:  # Process is still running
                        print(f"✅ Successfully started with: {cmd}")
                        print("🌐 Check the output above for the URL (usually http://localhost:3000)")
                        return True
                    
                except Exception as e:
                    print(f"⚠️ {cmd} failed: {e}")
                    continue
            
            print("ℹ️ npm commands didn't work, falling back to Python server")
            return False
            
        except Exception as e:
            print(f"❌ npm launch failed: {e}")
            return False
    return False

# Try npm first, fall back to Python server
if not launch_with_npm():
    # The Python server is already launched above
    pass


## Frontend and NIM Management Commands

Use these cells to manage your NIM container and frontend UI:


In [ ]:
# Check status of both NIM container and frontend
import requests

print("🔍 SYSTEM STATUS CHECK")
print("="*50)

# Check NIM container
print("📦 NIM Container Status:")
import subprocess
nim_check = subprocess.run(['docker', 'ps'], capture_output=True, text=True)
if 'nim-server' in nim_check.stdout:
    print("✅ NIM container is running")
    !docker ps | grep nim-server
else:
    print("❌ NIM container is not running")

# Check NIM API health
print("\n🔗 NIM API Health:")
try:
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✅ NIM API is responding")
    else:
        print(f"⚠️ NIM API returned status: {response.status_code}")
except:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        print("✅ NIM service is responding")
    except:
        print("❌ NIM API is not accessible")

# Check frontend
print("\n🌐 Frontend Status:")
try:
    response = requests.get("http://localhost:3000", timeout=5)
    print("✅ Frontend is running on http://localhost:3000")
except:
    print("❌ Frontend is not accessible on http://localhost:3000")

print("\n📊 Port Usage:")
!netstat -an | grep -E ":(3000|8000)" | grep LISTEN || echo "No services found on ports 3000 or 8000"


In [ ]:
# Restart frontend UI (if needed)
import os

print("🔄 Restarting frontend UI...")
print("Note: This will restart the frontend on http://localhost:3000")

# Stop any existing processes on port 3000
!lsof -ti:3000 | xargs kill -9 2>/dev/null || echo "No processes found on port 3000"

# Wait a moment
import time
time.sleep(2)

# Restart the frontend
os.chdir('prompt-to-trellis')

# Try different startup methods
def restart_frontend():
    if os.path.exists('package.json'):
        print("📦 Trying npm start...")
        !npm start &
    else:
        print("🐍 Starting with Python server...")
        !python -m http.server 3000 &

restart_frontend()

print("✅ Frontend restart initiated")
print("🌐 Open http://localhost:3000 in your browser")


In [ ]:
# Stop all services (NIM container and frontend)
import subprocess

print("🛑 SHUTTING DOWN ALL SERVICES")
print("="*50)

# Stop frontend (kill any process on port 3000)
print("🌐 Stopping frontend UI...")
try:
    subprocess.run(['lsof', '-ti:3000'], capture_output=True, text=True, check=True)
    !lsof -ti:3000 | xargs kill -9
    print("✅ Frontend stopped")
except:
    print("ℹ️ No frontend process found on port 3000")

# Stop and remove NIM container
print("\n📦 Stopping NIM container...")
!docker stop nim-server 2>/dev/null && echo "✅ NIM container stopped" || echo "ℹ️ NIM container was not running"
!docker rm nim-server 2>/dev/null && echo "✅ NIM container removed" || echo "ℹ️ NIM container was not found"

# Show final status
print("\n📊 Final Status:")
!echo "Docker containers:" && docker ps | grep nim-server || echo "No NIM container running"
!echo "Port 3000 processes:" && lsof -i:3000 || echo "No processes on port 3000"
!echo "Port 8000 processes:" && lsof -i:8000 || echo "No processes on port 8000"

print("\n🎉 All services stopped successfully!")
print("💡 Run the deployment cells again to restart everything")


## 🎯 Complete Workflow Summary

### Your Trellis NIM + Frontend UI is now fully integrated! 

**What was added:**
1. **Frontend Repository Integration** - Your Lovable-designed UI from `https://github.com/Soragi/prompt-to-trellis.git`
2. **NIM Service Bridge** - JavaScript integration layer (`nim-service.js`) 
3. **Configuration Management** - Automatic setup (`nim-config.json`)
4. **Multi-Server Setup** - NIM on port 8000, Frontend on port 3000

### **Quick Start Guide:**
1. **Deploy NIM**: Run cells 4-8 to start the Trellis NIM container
2. **Launch Frontend**: Run cells 14-18 to clone and start the UI
3. **Generate Models**: Open http://localhost:3000 and start prompting!

### **Management Commands:**
- **Status Check**: Cell 20 - Check both services
- **Restart Frontend**: Cell 21 - Restart UI if needed  
- **Stop Everything**: Cell 22 - Clean shutdown

### **Integration Features:**
- ✅ **Direct NIM Communication** - Frontend calls NIM API directly
- ✅ **Real-time 3D Viewer** - Generated .glb files display immediately  
- ✅ **CORS Handling** - Proper cross-origin setup
- ✅ **Error Management** - Graceful fallbacks and status checking
- ✅ **Auto-configuration** - No manual API endpoint setup needed

### **Troubleshooting:**
- If frontend won't load → Run cell 21 (restart frontend)
- If NIM not responding → Check cell 20 (status check)  
- If ports conflict → Modify port numbers in the launch cells
- If generation fails → Verify NIM container logs: `docker logs nim-server`

**🎉 You're all set! Happy 3D generating!**


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


In [ ]:
# List generated GLB files
!ls -la *.glb
